In [1]:
import pandas as pd
import os
import regex as re
import string
import numpy as np
import collections

import contractions
import spacy
#from spacy.tokenizer import Tokenizer
import en_core_web_sm
import nltk
#from nltk.stem import WordNetLemmatizer
from nltk import bigrams
#from sklearn.base import TransformerMixin, BaseEstimator
#import sys



In [2]:
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [3]:


#nlp = spacy.load('en', disable=['parser', 'ner'])
#nlp = en_core_web_sm.load()
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# #alter default spacy stopwords so 'not' isn't considered a stop word

    

# #Should hopefully deal with contractions
# special_cases = {":)": [{"ORTH": ":)"}]}


    
    
# def custom_tokenizer(nlp, custom_stop_words):
    
#     for w in customize_stop_words:
#         nlp.vocab[w].is_stop = False
        
#     return Tokenizer(nlp.vocab, rules=special_cases)

# nlp.tokenizer = custom_tokenizer(nlp, customize_stop_words)

In [4]:
# nlp = en_core_web_sm.load()

# customize_stop_words = {'not':False, 'yeah':True}

In [5]:
#https://www.kaggle.com/balatmak/text-preprocessing-steps-and-universal-pipeline

nlp = en_core_web_sm.load()

customize_stop_words = {'not':False, 'yeah':True}


for w,val in customize_stop_words.items():
    nlp.vocab[w].is_stop = val


class TextPreprocessor(object):
    def __init__(self):
        """
        TBD
        """

    def preprocess_df(self, df):
        return df.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        doc = nlp(text)
        removed_punct = self._remove_punct(doc)
        removed_stop_words = self._remove_stop_words(removed_punct)
        return self._lemmatize(removed_stop_words)

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if not t.is_stop]

    def _lemmatize(self, doc):
        return ' '.join([t.lemma_ for t in doc])



In [6]:
os.chdir("/Users/amywinecoff/Documents/CITP/Research/Github/ai-dialogues/startup-study/data/docs")

In [7]:
#replace some words so all texts refer to them the same way throughout
lookp_dict = {
    "artificial intelligence" : "ai", 
    "machine learning" : "ml",
    }

In [8]:
pid = 'P1'
industry='Marketing'


transcript = open('{}.txt'.format(pid), "r", encoding='utf-8').read().lower()
transcript = contractions.fix(transcript)
transcript_df = pd.DataFrame(transcript.split('*'), columns=['quote'])
transcript_df['pid']=pid
transcript_df['industry']=industry

transcript_df.head()



,quote,pid,industry
0,"sure, so at companyname we are a software tech...",P1,Marketing
1,"yeah. all right, so we do small promotion and ...",P1,Marketing
2,yeah. so once a month we co-host a startup int...,P1,Marketing
3,"yeah, so on top of our own solution, we use ai...",P1,Marketing
4,"yeah, sure. there is actually one blog post th...",P1,Marketing


In [9]:
transcript_df['pp_text']=(TextPreprocessor().preprocess_df(transcript_df['quote']))
#text.loc[0]['quote']
transcript_df.head()

,quote,pid,industry,pp_text
0,"sure, so at companyname we are a software tech...",P1,Marketing,sure companyname software technology company b...
1,"yeah. all right, so we do small promotion and ...",P1,Marketing,right small promotion customer feedback featur...
2,yeah. so once a month we co-host a startup int...,P1,Marketing,month co host startup interview usually startu...
3,"yeah, so on top of our own solution, we use ai...",P1,Marketing,solution use ai different way use analytic use...
4,"yeah, sure. there is actually one blog post th...",P1,Marketing,sure actually blog post co publish ai tool let...


In [10]:
#transcript=transcript_df.groupby(['pid'])['pp_text'].transform(lambda x : ' '.join(x))


# transcript_df['all_text'] = transcript_df[['pp_text','pid']].groupby(['pid'])['pp_text'].transform(lambda x: ','.join(x))
# #transcript_df[['pid','pp_text']].drop_duplicates()

# print(len(transcript_df.loc[0]['all_text']))


all_text = transcript_df[['pid', 'pp_text']].groupby(['pid'], as_index = False).agg({'pp_text': ' '.join})

all_text


,pid,pp_text
0,P1,sure companyname software technology company b...


In [11]:
# counts_no_urls = collections.Counter(all_words_no_urls)

# Create counter
word_counts = collections.Counter(all_text.loc[0]['pp_text'].split())

word_counts.most_common(50)

[('like', 48),
 ('not', 45),
 ('think', 40),
 ('...', 35),
 ('marketing', 33),
 ('startup', 33),
 ('actually', 32),
 ('help', 30),
 ('sure', 28),
 ('kind', 27),
 ('use', 26),
 ('lot', 25),
 ('great', 25),
 ('ai', 24),
 ('team', 23),
 ('tool', 23),
 ('let', 20),
 ('basically', 20),
 ('different', 19),
 ('time', 18),
 ('thing', 18),
 ('try', 18),
 ('customer', 17),
 ('plan', 17),
 ('program', 17),
 ('way', 17),
 ('work', 16),
 ('want', 16),
 ('specific', 16),
 ('information', 15),
 ('week', 15),
 ('content', 15),
 ('hey', 15),
 ('question', 15),
 ('focus', 15),
 ('partnership', 15),
 ('product', 14),
 ('feedback', 13),
 ('accelerator', 13),
 ('people', 13),
 ('datum', 13),
 ('share', 12),
 ('analytic', 11),
 ('need', 11),
 ('example', 11),
 ('sale', 11),
 ('get', 11),
 ('investor', 11),
 ('company', 10),
 ('base', 10)]

In [12]:
#all_text.loc[0]['pp_text']

'sure companyname software technology company base thesouthwest year launch publicly automate marketing planning work startup smb agency enterprise marketing team usually marketer use solution automate market research identify ideal customer profile help customer decision journey tool ai base use course use tool ai base ... help save time identify growth opportunity startup smb enterprise marketing team agency use solution help scale fast let right consider current resource marketing plan resource tool multiply 20 \n\n\n right small promotion customer feedback feature update platform publish share information startup interview prior clubhouse check social schedule check analytic weekly analytic check team plan week talk customer demo check upcoming event preparation kind thing let think main thing \n\n\n month co host startup interview usually startup story early stage startup like indie hacker maker small team early stage not fund growth traction talk grow startup bootstrappe talk bac